<a href="https://colab.research.google.com/github/Noertri/scraper-nusabali/blob/main/scraper_nusabali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Menginstall package yang diperlukan

In [ ]:
!pip install --upgrade -r requirements.txt

In [ ]:
!pip list